In [219]:
import pandas as pd

In [220]:
db = pd.read_excel('fitness.xlsx')

In [221]:
db.head()

,вопрос,группа
0,Последнее время даже в обычных супермаркетах с...,другое
1,"Что лучше посещать – тренажерные залы, бассейн...",тренажерный зал
2,Как быстро я начну меняться в лучшую сторону?,тренажерный зал
3,"У меня избыточный вес, но я стесняюсь занимать...",тренажерный зал
4,"Если тренироваться, но совершенно не следить з...",питание


In [222]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [223]:
import re
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [224]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
noise = stopwords.words('russian')
noise = noise+['это', 'тыс', 'млн', 'млрд', 'также']
mystoplemmas = ['который','прошлый','сей', 'свой', 'наш','ваш','очень', 'мочь','этот', 'такой', ' ', '\n']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\volko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [225]:
def dataprep(inf):
    dp = [words_only(x.lower()) for x in inf]
    dp = [x.replace(' ',',').split(',') for x in dp]
    long_words = []
    for j in tqdm(dp):
        long_words.append([x for x in j if len(x)>1])
    latterm_wo_stopw = []
    for k in tqdm(long_words):
        latterm_wo_stopw.append(' '.join([x for x in k if x not in noise]))
    return latterm_wo_stopw

In [226]:
from rnnmorph.predictor import RNNMorphPredictor
predictor=RNNMorphPredictor(language='ru')

In [227]:
def lem(a):
    res = []
    for i in tqdm(a):
        i = i.split(' ')
        forms = predictor.predict(i)
        res.append([word.normal_form for word in forms])
    return res

In [228]:
def lem_wo_stop(a):
    latterm = []
    for i in tqdm(a):
        latterm.append(' '.join([x for x in i if x not in mystoplemmas]))
    return latterm

In [229]:
from tqdm import tqdm

In [230]:
data = dataprep(db['вопрос'])

100%|█████████████████████████████████████████████████████████████████████████████| 281/281 [00:00<00:00, 11442.05it/s]


In [231]:
data = lem(data)

100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [00:06<00:00, 46.30it/s]


In [232]:
data = lem_wo_stop(data)

100%|████████████████████████████████████████████████████████████████████████████| 281/281 [00:00<00:00, 127072.71it/s]


In [233]:
tfidf = TfidfVectorizer(ngram_range=(1,2))

In [234]:
tfidf_data = tfidf.fit_transform(data)

In [235]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y=le.fit_transform(db['группа'])

In [236]:
from sklearn.model_selection import train_test_split

In [237]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_data, y, stratify=y, test_size=0.15, random_state=42)

In [238]:
from  sklearn.linear_model import LogisticRegression

In [239]:
model1=LogisticRegression()

In [240]:
model1.fit(X_train, y_train)

C:\Users\volko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\volko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [241]:
from sklearn.metrics import *

In [242]:
y_pred=model1.predict(X_test)
accuracy_score(y_pred, y_test)

0.5348837209302325

In [243]:
from sklearn.ensemble import RandomForestClassifier

In [244]:
model2=RandomForestClassifier(n_estimators=200, max_depth=50)

In [245]:
model2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [246]:
y_pred=model2.predict(X_test)
accuracy_score(y_pred, y_test)

0.6046511627906976

In [247]:
import xgboost as xgb

In [248]:
xgb_model = xgb.XGBClassifier(n_estimators=200,  max_depth=50)

In [249]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=50, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [250]:
y_pred=xgb_model.predict(X_test)
accuracy_score(y_pred, y_test)

0.5348837209302325

In [251]:
from sklearn.decomposition import TruncatedSVD

In [252]:
svd = TruncatedSVD(n_components=14, n_iter=30, random_state=42)
svd_tfidf_train=svd.fit_transform(X_train) 
svd_tfidf_test=svd.transform(X_test) 

In [253]:
model2.fit(svd_tfidf_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [254]:
y_pred=model2.predict(svd_tfidf_test)
accuracy_score(y_pred, y_test)

0.4883720930232558